# 📈 Scraping de Notícias sobre o Mercado de Ações com Selenium e Pandas

Este notebook realiza a coleta automatizada de notícias sobre o mercado de ações usando o Selenium para navegação e captura de dados dinâmicos, além de processamento dos dados extraídos com Pandas.

### Etapas:
1. **Iniciar o WebDriver**: Inicia o Selenium WebDriver para controlar o navegador e acessar as páginas de interesse.
2. **Captura de Notícias**: Navega para a página de notícias sobre o mercado de ações e coleta os itens de notícias.
3. **Extração de Dados**: Para cada notícia, extrai informações como título, descrição, link e data de publicação.
4. **Conversão de Data**: Converte os tempos relativos de publicação para um formato de data legível.
5. **Armazenamento em CSV**: Organiza os dados extraídos em um DataFrame e os salva em um arquivo CSV para análise posterior.

🚀 **Objetivo**: Automatizar a extração de notícias sobre o mercado financeiro e organizar os dados para futuras análises.

In [1]:
# !pip install selenium pandas

In [2]:
import os
import time
import pandas as pd
import textwrap
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime

In [ ]:
# Função para iniciar o webdriver
def start_driver():
    """
    Inicia o WebDriver do Selenium com as opções especificadas para navegação.

    Retorna:
        webdriver.Chrome: Instância do driver do Chrome configurada para navegação.
    """
    options = Options()
    # options.add_argument('--headless')  # Executa o navegador em modo headless (sem interface gráfica)
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Chrome(options=options)
    return driver

driver = start_driver()

In [ ]:
def find_stock_market_news(driver):    
    """
    Coleta notícias sobre o mercado de ações da página "https://finance.yahoo.com/topic/stock-market-news/".

    Args:
        driver (webdriver.Chrome): Instância do WebDriver Selenium para navegar e coletar os dados.

    Retorna:
        list: Lista de strings HTML contendo os itens de notícias encontrados.
    """
    url = "https://finance.yahoo.com/topic/stock-market-news/"
    driver.get(url)
    time.sleep(5)  # Aguarde o carregamento inicial da página
    
    news_items = []
    scroll_pause_time = 2  # Tempo de pausa entre rolagens
    max_attempts = 40  # Número máximo de tentativas de rolagem
    max_items = 100  # Limite de itens a capturar

    for _ in range(max_attempts):
        # Encontre a lista de notícias e os itens atualmente visíveis
        news_list = driver.find_element(By.CSS_SELECTOR, "ul.My\\(0\\).P\\(0\\).Wow\\(bw\\).Ov\\(h\\)")
        new_items = news_list.find_elements(By.TAG_NAME, "li")

        # Verifique se novos itens foram carregados e adicione-os
        if len(new_items) > len(news_items):
            news_items = new_items
            print(f"Found {len(news_items)} items")
        
        # Interrompe o loop se o limite de itens for alcançado
        if len(news_items) >= max_items:
            news_items = news_items[:max_items]  # Garante exatamente 50 itens
            break

        # Role um pouco para baixo
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(scroll_pause_time)

    # Converte os itens de notícias em strings HTML
    news_items_html = [item.get_attribute('outerHTML') for item in news_items]

    return news_items_html

news_items = find_stock_market_news(driver)
driver.quit()


Found 13 items
Found 25 items
Found 38 items
Found 50 items
Found 63 items
Found 75 items
Found 88 items
Found 100 items


In [5]:
news_items

['<li class="js-stream-content Pos(r)"><div class="Py(14px) Pos(r)" data-test-locator="mega"><div class="Cf"><div class="Fl(start) Pos(r) Mt(2px) W(26.5%) Maw(220px)"><div class="H(0) Ov(h) Bdrs(2px)" style="padding-bottom:56%"><img srcset="https://s.yimg.com/uu/api/res/1.2/vI0oSauBMer7HU42PSRJLw--~B/Zmk9c3RyaW07aD0xMjM7cT04MDt3PTIyMDthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/business_insider_articles_888/7de726e134a4a2b3892b65eaa206937b.cf.webp 1x,https://s.yimg.com/uu/api/res/1.2/G_xMTfeomeD9fdq5E8jTMA--~B/Zmk9c3RyaW07aD0yNDY7cT04MDt3PTQ0MDthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/business_insider_articles_888/7de726e134a4a2b3892b65eaa206937b.cf.webp 2x" class=" W(100%) Trsdu(0s)! Bdrs(2px)" alt="" data-status="LOADED" src="https://s.yimg.com/uu/api/res/1.2/vI0oSauBMer7HU42PSRJLw--~B/Zmk9c3RyaW07aD0xMjM7cT04MDt3PTIyMDthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/business_insider_articles_888/7de726e134a4a2b3892b65eaa206937b.cf.webp"></div></div><div class="Ov(h) Pe

In [ ]:
from datetime import datetime, timedelta
import re

def convert_relative_time(relative_time_str):
    """
    Converte uma string de tempo relativo (ex: '5 minutes ago') para um timestamp absoluto.

    Args:
        relative_time_str (str): String representando o tempo relativo.

    Retorna:
        str: Data e hora no formato 'YYYY-MM-DD HH:MM:SS'.
    """
    # Obtemos o horário atual
    now = datetime.now()

    # Expressões regulares para identificar a quantidade e unidade de tempo
    match = re.match(r"(\d+)\s+(minute|hour|day)s?\s+ago", relative_time_str)
    if not match:
        raise ValueError("Formato de tempo inválido")

    amount = int(match.group(1))
    unit = match.group(2)

    # Calcula a diferença de tempo com base na unidade
    if unit == "minute":
        time_delta = timedelta(minutes=amount)
    elif unit == "hour":
        time_delta = timedelta(hours=amount)
    elif unit == "day":
        time_delta = timedelta(days=amount)
    else:
        raise ValueError("Unidade de tempo não suportada")

    # Calcula o horário desejado
    converted_time = now - time_delta

    # Retorna o horário no formato "YYYY-MM-DD HH:MM:SS"
    return converted_time.strftime("%Y-%m-%d %H:%M:%S")

In [7]:
# # Função para converter tempo para timestamp
# def convert_time_to_timestamp(time_text):
#     # Aqui você pode definir diferentes regras dependendo do formato do tempo (ex: "7 minutes ago", "2 hours ago", etc.)
#     if 'minute' in time_text:
#         minutes_ago = int(time_text.split()[0])
#         return datetime.now() - pd.to_timedelta(minutes_ago, unit='m')
#     elif 'hour' in time_text:
#         hours_ago = int(time_text.split()[0])
#         return datetime.now() - pd.to_timedelta(hours_ago, unit='h')
#     # Adicione mais regras conforme necessário para outros formatos de tempo
#     else:
#         return pd.to_datetime(time_text)

In [ ]:
def extract_text_string(html_string, keyword, start_char, end_char, division_index: int=1):
    """
    Extrai um trecho de texto de uma string HTML com base em uma palavra-chave e os caracteres delimitadores.

    Args:
        html_string (str): A string HTML de onde extrair o texto.
        keyword (str): Palavra-chave para localizar o ponto de extração.
        start_char (str): Caractere de início do trecho a ser extraído.
        end_char (str): Caractere de fim do trecho a ser extraído.
        division_index (int, opcional): Índice para dividir a string HTML (padrão é 1).

    Retorna:
        str: O texto extraído, ou None se a extração falhar.
    """
    try:
        # Divide o HTML pela palavra-chave
        parts = html_string.split(keyword)
        # print(f"Parts[1] ({len(parts)}) :", parts[1])
        
        # Verifica se a palavra-chave está no HTML
        if len(parts) < 2:
            print(f'({len(parts)})\n\n({keyword})\n{parts}\n\n')
            return None
        
        # Pega a segunda parte da divisão
        text_part = parts[division_index]
        
        # Encontra o índice do caractere inicial
        start_index = text_part.find(start_char)
        
        # Verifica se o caractere inicial foi encontrado
        if start_index == -1:
            return None
        
        # Ajusta o índice inicial para começar após o caractere inicial
        start_index += len(start_char)
        
        # Encontra o índice do caractere final a partir do índice inicial
        end_index = text_part.find(end_char, start_index)
        
        # Verifica se o caractere final foi encontrado
        if end_index == -1:
            return None
        
        # Extrai o texto entre o caractere inicial e o caractere final
        extracted_text = text_part[start_index:end_index]
        
        return extracted_text.strip()
    except Exception as e:
        print(f"Erro ao extrair texto: {e}")
        return None

In [ ]:
# Função para extrair dados de um item de notícia
def extract_news_item_data(item):
    """
    Extrai dados (link, título, descrição, data de publicação) de um item de notícia.

    Args:
        item (str): String HTML representando um item de notícia.

    Retorna:
        dict: Dicionário contendo 'Link', 'Title', 'Description' e 'PublishDate'.
    """
    # print(f'Item:\n\n{textwrap.indent(item.get_attribute("innerHTML"), prefix='    ')}\n\n')
    try:        
        # Extraindo o link do artigo
        link = extract_text_string(item, "href=", '"', '"')
        print(f'Article Link: {link}')

        # Extraindo o título do artigo
        title = extract_text_string(item, 'class="StretchedBox"></u', ">", "<")
        print(f'Article Title: {title}')

        # Extraindo a descrição do artigo
        description = extract_text_string(item, 'class="Fz(14px) Lh(19px)', ">", "<")
        print(f'Article Description: {description}')

        # Extraindo a data de publicação
        publish_date = extract_text_string(item, 'class="Mx(4px)">•</i><span', '>', '<')
        publish_date = convert_relative_time(publish_date)
        print(f'Publish Date: {publish_date}')

        return {
            'Link': link,
            'Title': title,
            'Description': description,
            'PublishDate': publish_date
        }
    except Exception as e:
        print(f"Erro ao processar item: {e}")
        return None

In [ ]:
# Função para extrair os dados das notícias
def extract_news_data(news_items):
    """
    Extrai dados de uma lista de itens de notícias e os organiza em um DataFrame.

    Args:
        news_items (list): Lista de strings HTML contendo os itens de notícias.

    Retorna:
        pd.DataFrame: DataFrame contendo os dados extraídos (link, título, descrição, data de publicação).
    """
    # Lista para armazenar os dados
    news_data = []

    # Itera sobre os itens de notícias
    for item in news_items:
        if item.startswith('<li class="js-stream-content Pos(r)">'):
            news_item_data = extract_news_item_data(item)
            if news_item_data and all(news_item_data[param] is not None for param in ['Link', 'Title', 'Description', 'PublishDate']):
                news_data.append(news_item_data)

    # Cria um DataFrame com os dados extraídos
    news_df = pd.DataFrame(news_data)

    return news_df

news_df = extract_news_data(news_items)
news_df.to_csv('news_data.csv', index=False)
print("Dados salvos em 'news_data.csv'")

Article Link: https://finance.yahoo.com/news/hint-3rd-quarter-earnings-calls-000251106.html
Article Title: A hint in 3rd-quarter earnings calls suggests S&amp;P 500 corporate profits are about to boom, Bank of America says
Article Description: Notable surges in the word "bottom" being mentioned on earnings calls in 2009 and 2020 were soon followed by a corporate-profit surge, a note said.
Publish Date: 2024-10-29 21:03:32
Article Link: https://mrwisebuyer.com/advertorial-knee-elite-br/?lang=pt-br&amp;utm_source=taboola&amp;utm_medium=cpc&amp;utm_content=Joelho+desgastado%3F+Isso+pode+rejuvenescer+seus+joelhos+em+17+anos%21&amp;utm_campaign=Knee+Elite+BR+Desktop+V3+%28B%3A28+T%3A16%29&amp;utm_term=Joelho+desgastado%3F+Isso+pode+rejuvenescer+seus+joelhos+em+17+anos%21&amp;ad_id=4062111760&amp;utm_id=37969559&amp;site_id=1551783&amp;placement=yahoo-finances&amp;device=Desktop&amp;thumbnail=http%3A%2F%2Fcdn.taboola.com%2Flibtrc%2Fstatic%2Fthumbnails%2F2693991df64af6246b39fdef2ba05f9e.jpg&a

In [ ]:
def scrape_and_save_news():
    """
    Função principal que executa o scraping das notícias, processa os dados e os salva em um arquivo CSV.

    Salva os dados em 'news_data.csv', adicionando dados se o arquivo já existir.
    """
    driver = start_driver()
    news_df = extract_news_data(driver)
    driver.quit()  # Fecha o navegador
    
    # Verifica se o arquivo já existe
    file_exists = os.path.isfile('news_data.csv')
    
    # Salva o DataFrame em um arquivo CSV, adicionando dados se o arquivo já existir
    news_df.to_csv('news_data.csv', mode='a', header=not file_exists, index=False)
    print("Dados adicionados em 'news_data.csv'")
    
# Chama a função principal
# scrape_and_save_news()